In [1]:
import pymmcore
import os.path
import sys
import time

Installation of packages with pip (example for `nidaqmx`): `C:\Users\TCSPC\anaconda3\envs\scopecontrol\Scripts\pip3.exe install --user nidaqmx`

In [3]:
mm_dir = r"C:\Program Files\Micro-Manager-2.0.1"

mmc = pymmcore.CMMCore()

In [4]:
pymmcore.CMMCore().getAPIVersionInfo()

'Device API version 70, Module API version 10'

In [5]:
mmc.setDeviceAdapterSearchPaths([mm_dir])
mmc.loadSystemConfiguration(os.path.join(mm_dir, "MMConfig_xyz_shutter_filter2.cfg"))

# mmc.snapImage()
# mmc.getImage()

In [6]:
mmc.getLoadedDevices()

('COM1',
 'COM2',
 'COM8',
 'PIZStage',
 'XYStage',
 'SutterHub',
 'Wheel-A',
 'Shutter-A',
 'Core')

# PyMMCore control

## get/set XYZ

In [7]:
def displayDeviceProperties(devicename):
    for prop in mmc.getDevicePropertyNames(devicename):
        print(prop, ": ",  mmc.getProperty(devicename, prop))
        
displayDeviceProperties("XYStage")

Acceleration :  100
Description :  Prior XY stage driver adapter
MaxSpeed :  100
Port :  COM1
SCurve :  100
StepSizeX_um :  1.0000
StepSizeY_um :  1.0000
TransposeMirrorX :  0
TransposeMirrorY :  0


In [8]:
# XY stage
# mmc.setXYPosition(-5000.00,465.00)
mmc.getXYStageDevice(), mmc.getXYPosition()

('XYStage', [2535.0, 1756.0])

In [9]:
# Z piezo
# mmc.setPosition(11)
mmc.getPosition()

42.8777

## get/set shutter/filter wheel position

### filter wheel

In [12]:
for prop in mmc.getDevicePropertyNames("Wheel-A"):
    print(prop, ": ",  mmc.getProperty("Wheel-A", prop))

Busy :  0
ClosedPosition :  
Delay_ms :  0.0000
Description :  Sutter Lambda Filter Wheel
Label :  Filter-0
Name :  Wheel-A
State :  0
State Change Speed :  3


In [13]:
wheel_values = mmc.getStateLabels("Wheel-A")
wheel_values

('Filter-0',
 'Filter-1',
 'Filter-2',
 'Filter-3',
 'Filter-4',
 'Filter-5',
 'Filter-6',
 'Filter-7',
 'Filter-8',
 'Filter-9')

In [28]:
mmc.setStateLabel("Wheel-A", "Filter-0")
mmc.getStateLabel("Wheel-A")

'Filter-0'

In [33]:
mmc.setStateLabel("Wheel-A", 'Filter-0')

### shutter

In [2]:
mmc.setShutterOpen(True)
mmc.getShutterOpen()

NameError: name 'mmc' is not defined

In [41]:
mmc.setShutterOpen(False)
mmc.getShutterOpen()

False

# Insight control

In [42]:
import pyvisa

In [46]:
rm = pyvisa.ResourceManager()

resource_list = rm.list_resources()

print(resource_list)

('TCPIP0::140.247.108.15::inst0::INSTR', 'ASRL1::INSTR', 'ASRL2::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL8::INSTR', 'ASRL11::INSTR')


In [47]:
with rm.open_resource(resource_list[-1]) as my_instrument:
    my_instrument.baud_rate = 115200
    my_instrument.write_termination = '\r'
    # my_instrument.write("*IDN?")
    print(my_instrument.query_ascii_values(r"MODE?", converter="s")[0])
    print(my_instrument.query_ascii_values(r"SHUTter?", converter="s")[0])
    print(my_instrument.query_ascii_values(r"READ:POWer?", converter="s")[0])
    
    # my_instrument.write_ascii_values(r"SHUTter 0", "")
    # my_instrument.write_ascii_values(r"OFF", "")
    time.sleep(1)
    print(my_instrument.query_ascii_values(r"READ:POWer?", converter="s")[0])
    
#     while True:
#         print(my_instrument.read_bytes(1))


RUN

0

0.00

0.00



# ThorLabs Kinesis motorized mounts

refer to documentation at https://pylablib.readthedocs.io/en/latest/devices/Thorlabs_kinesis.html

Note that if LabVIEW program was opened, you must reboot the motor controllers in order to detect them with ppylablib.

In [77]:
from pylablib.devices import Thorlabs

In [78]:
kinesis_devices = Thorlabs.list_kinesis_devices()
sync_stage, hwp_stage = kinesis_devices[0], kinesis_devices[1]
print(kinesis_devices)

[('27253344', 'Brushed Motor Controller'), ('27004338', 'Brushed Motor Controller')]


In [97]:
stage.close()
stage = Thorlabs.KinesisMotor('27004338', scale="stage")
print(dir(stage), end="\n\n")
print(stage.get_full_status(), end="\n\n")
print(stage.is_homed(), end="\n\n")
print(stage.get_position(), end="\n\n")
stage.close()

['CommData', 'CommShort', 'Error', 'NoParameterCaller', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_device_variable', '_add_info_variable', '_add_parameter_class', '_add_settings_variable', '_add_status_variable', '_autodetect_stage', '_bg_msg_counters', '_calculate_scale', '_call_without_parameters', '_connection_parameters', '_d2p', '_device_SN', '_device_var_ignore_error', '_device_vars', '_device_vars_order', '_forward_positive', '_get_adc_inputs', '_get_connection_parameters', '_get_device_model', '_get_device_variables', '_get_gen_move_parameters', '_get_homing_parameters', '_get_jog_parameters', '_get_limit_switch_parameters', '_get_position'

In [96]:
with Thorlabs.KinesisMotor("27004338", scale="stage") as hwp_stage:
    print(hwp_stage.get_full_status())
    if not hwp_stage.is_homed():
        
    # # print(hwp_sample.get_homing_parameters())
    # # time.sleep(3)
    hwp_stage.home()
    hwp_stage.wait_for_home()
    hwp_stage.move_by(10)
    hwp_stage.move_to(6)
    hwp_stage.wait_for_stop()
    print(hwp_stage.get_position())
    # hwp_sample.move_to(100000)

C:\Users\TCSPC\anaconda3\envs\scopecontrol_napari\lib\site-packages\pylablib\devices\Thorlabs\kinesis.py:734: UserWarning: can't recognize the stage name STAGE; assuming step scale of 1
  warnings.warn("can't recognize the stage name {}; assuming step scale of 1".format(stage))


{'velocity_parameters': TVelocityParams(min_velocity=0.0, acceleration=19252.18384712934, max_velocity=19195.98877429962), 'jog_parameters': TJogParams(mode='step', step_size=9598, min_velocity=0.0, acceleration=28812.792152166363, max_velocity=28794.988989830017, stop_mode='profiled'), 'homing_parameters': THomeParams(home_direction='reverse', limit_switch='reverse', velocity=19195.98877429962, offset_distance=7679), 'gen_move_parameters': TGenMoveParams(backlash_distance=1920), 'limit_switch_parameters': TLimitSwitchParams(hw_kind_cw='make_home', hw_kind_ccw='ignore', hw_swapped=False, sw_position_cw=1920, sw_position_ccw=1920, sw_kind='ignore'), 'position': 59, 'status': ['homed', 'enabled']}
True
6


# bh trigger

references:
https://stackoverflow.com/questions/49951166/nidaqmx-co-channels-cant-write-sample
https://github.com/ni/nidaqmx-python/blob/master/nidaqmx/_task_modules/co_channel_collection.py#L160

In [1]:
import nidaqmx
import nidaqmx.system
from nidaqmx.types import CtrTime

In [2]:
system = nidaqmx.system.System.local()

In [3]:
system.driver_version

DriverVersion(major_version=18, minor_version=5, update_version=0)

In [4]:
for device in system.devices:
    print(device)

Device(name=Dev1)


In [6]:
with nidaqmx.Task() as task:
    task.co_channels.add_co_pulse_chan_time(counter="Dev1/ctr1", low_time=0.010, high_time=0.010)
    task.start()